In [2]:
import numpy as np
from networkx.algorithms.approximation.distance_measures import diameter


class Config:
    FIREBASE_CRED_PATH = '../.local/firebase/firebase-adminsdk.json'
    FIREBASE_CONFIG_PATH = '../.local/firebase/firebase_config.json'
    
conf = Config()

In [2]:
import firebase_admin
from firebase_admin import credentials, firestore


class FirebaseService:
    DATASET_COLLECTION = 'gaze_data'
    GAZE_COLLECTION = 'gaze_points'
    def __init__(self):
        if not firebase_admin._apps:
            cred = credentials.Certificate(Config.FIREBASE_CRED_PATH)
            firebase_admin.initialize_app(cred)
        self.db = firestore.client()
        
    def get_documents(self):
        collection_ref = self.db.collection(self.DATASET_COLLECTION)
        docs = collection_ref.stream()
        
        docs_set = []
        for i, doc in enumerate(docs):
            doc_dict = doc.to_dict()
            doc_dict['id'] = doc.id
            docs_set.append(doc_dict)
            
        return docs_set

    def get_gaze_from_document(self, document_id):
        collection_ref = self.db.collection(self.DATASET_COLLECTION)
        doc_ref = collection_ref.document(document_id)
        collection = doc_ref.collection(self.GAZE_COLLECTION)
        docs = collection.stream()
        docs = [doc.to_dict() for doc in docs]
        return docs

In [3]:
import pprint

firebase_service = FirebaseService()
data_docs = firebase_service.get_documents()

docs_0 = [doc for doc in data_docs if doc['task'] == 'No Interaction']
docs_1 = [doc for doc in data_docs if doc['task'] == 'Interact']
print(f'No Interaction: {len(docs_0)}')
print(f'Interaction: {len(docs_1)}')

gaze_0 = firebase_service.get_gaze_from_document(docs_0[2]['id'])
gaze_1 = firebase_service.get_gaze_from_document(docs_1[2]['id'])
pprint.pprint(gaze_0[0])


KeyboardInterrupt



In [ ]:
import matplotlib.pyplot as plt

def get_diameter_values(gaze_points):
    diameter_values = []
    for gaze_point in gaze_points:
        if len(gaze_point['base_data']) != 2:
            diameter_values.append([np.nan, np.nan])
            continue
        d_left = gaze_point['base_data'][0]['diameter']
        d_right = gaze_point['base_data'][1]['diameter']
        diameter_values.append([d_left, d_right])
    return np.array(diameter_values)

def get_timestamps(gaze_points):
    timestamps = []
    for gaze_point in gaze_points:
        timestamps.append(gaze_point['timestamp'])
    return np.array(timestamps)



In [ ]:
means_0 = []
means_1 = []
for doc in docs_0:
    gaze = firebase_service.get_gaze_from_document(doc['id'])
    print(doc)
    if len(gaze) == 840:
        diameters = get_diameter_values(gaze)[:, 0]
        means_0.append(np.nanmean(diameters))

for doc in docs_1:
    gaze = firebase_service.get_gaze_from_document(doc['id'])
    print(doc)
    if len(gaze) == 840:
        diameters = get_diameter_values(gaze)[:, 0]
        means_1.append(np.nanmean(diameters))

In [ ]:
plt.scatter(range(len(means_0)), means_0, label='No Interaction')
plt.scatter(range(len(means_1)), means_1, label='Interaction')
plt.title('Mean Diameter values')
plt.legend()
plt.show()

In [ ]:
diameters_0 = get_diameter_values(gaze_0)[:, 0]
timestamps_0 = get_timestamps(gaze_0)

timestamps_0, diameters_0 = zip(*sorted(zip(timestamps_0, diameters_0)))
timestamps_0 = timestamps_0 - timestamps_0[0]

diameters_1 = get_diameter_values(gaze_1)[:, 0]
timestamps_1 = get_timestamps(gaze_1)

timestamps_1, diameters_1 = zip(*sorted(zip(timestamps_1, diameters_1)))
timestamps_1 = timestamps_1 - timestamps_1[0]

plt.plot(timestamps_0, diameters_0, label='No Interaction')
mean_diameter_0 = np.nanmean(diameters_0)
plt.axhline(y=mean_diameter_0, color='r', linestyle='--')

plt.plot(timestamps_1, diameters_1, label='Interaction')
mean_diameter_1 = np.nanmean(diameters_1)
plt.axhline(y=mean_diameter_1, color='g', linestyle='--')

plt.title('Mean (left, right) Diameter values')
plt.legend()
plt.show()